# Homework  
Data documentation: https://tidyr.tidyverse.org/reference/who.html

__Directions:__ Answer the three questions below. Create as many code/markdown blocks necessary to answer the questions.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px

df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/who.csv')

In [2]:
df.head()

,country,iso2,iso3,year,new_sp_m014,new_sp_m1524,new_sp_m2534,new_sp_m3544,new_sp_m4554,new_sp_m5564,...,newrel_m4554,newrel_m5564,newrel_m65,newrel_f014,newrel_f1524,newrel_f2534,newrel_f3544,newrel_f4554,newrel_f5564,newrel_f65
0,Afghanistan,AF,AFG,1980,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AF,AFG,1981,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AF,AFG,1982,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AF,AFG,1983,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AF,AFG,1984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df.drop(columns=['iso2', 'iso3'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7240 entries, 0 to 7239
Data columns (total 58 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       7240 non-null   object 
 1   year          7240 non-null   int64  
 2   new_sp_m014   3173 non-null   float64
 3   new_sp_m1524  3209 non-null   float64
 4   new_sp_m2534  3206 non-null   float64
 5   new_sp_m3544  3219 non-null   float64
 6   new_sp_m4554  3223 non-null   float64
 7   new_sp_m5564  3218 non-null   float64
 8   new_sp_m65    3209 non-null   float64
 9   new_sp_f014   3174 non-null   float64
 10  new_sp_f1524  3194 non-null   float64
 11  new_sp_f2534  3200 non-null   float64
 12  new_sp_f3544  3199 non-null   float64
 13  new_sp_f4554  3204 non-null   float64
 14  new_sp_f5564  3195 non-null   float64
 15  new_sp_f65    3197 non-null   float64
 16  new_sn_m014   1045 non-null   float64
 17  new_sn_m1524  1030 non-null   float64
 18  new_sn_m2534  1022 non-null 

## Question 1. We need to know what's included in this data.
- How many countries are present?  
- What's the timespan of the data?  
- Does each country have a row for every year present?  
- Which countries are missing years?  
- How many rows have at least 1 non-null values across the columns 3-57?

In [4]:
# total countries present
unique_countries=df['country'].nunique()

# timespan of the data
timespan = df['year'].min(), df['year'].max()

# does each country have a row for every year present?
missing_countries = []

#  number of unique years in the dataset
total_years = df['year'].nunique()

#  for loop to check each country
for country in df['country'].unique():
    country_years = df[df['country'] == country]['year'].nunique()
    if country_years < total_years:
        missing_countries.append(country)

# check if all countries have a row for each year
all_countries_have_all_years = len(missing_countries) == 0

# number of rows that have at least 1 non-null values across the columns 3-57
non_null_rows = df.iloc[:, 3:58].notnull().any(axis=1).sum()

# results
{
    "Total number of countries present": unique_countries,
    "Timespan of the data": timespan,
    "Does each country have a row for every year?": all_countries_have_all_years,
    "Countries missing years": missing_countries,
    "number of rows that have atleast 1 non-null value": non_null_rows
}

{'Total number of countries present': 219,
 'Timespan of the data': (1980, 2013),
 'Does each country have a row for every year?': False,
 'Countries missing years': ['Bonaire, Saint Eustatius and Saba',
  'Curacao',
  'Montenegro',
  'Netherlands Antilles',
  'Serbia',
  'Serbia & Montenegro',
  'Sint Maarten (Dutch part)',
  'South Sudan',
  'Timor-Leste'],
 'number of rows that have atleast 1 non-null value': 3484}

## Question 2: Wide data isn't usually helpful.
Convert this to a long format with the following columns:  
1. country  
2. year  
3. diagnosis method: mapping should be:

```
  {'newsp':'Pulmonary Smear (+)'
  ,'newsn':'Pulmonary Smear (-)'
  ,'newep':'Extrapulmonary'
  ,'newrel':'Relapse'}   
 ```

4. gender: male or female.   
5. age: lower_age - higher_age, e.g., 0-14  
6. number of cases  

__Note__: You can use `replace` (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html)

In [5]:
# mapping for diagnosis method
diagnosis_mapping = {
    'new_sp': 'Pulmonary Smear (+)',
    'new_sn': 'Pulmonary Smear (-)',
    'new_ep': 'Extrapulmonary',
    'newrel': 'Relapse'
}

# melting the dataset into long format
long_format_df = pd.melt(df,
                         id_vars=['country', 'year'],
                         var_name='temporary',
                         value_name='number_of_cases')

# extracting diagnosis method, gender, and age from the column names
long_format_df['diagnosis_method'] = long_format_df['temporary'].str[:6].map(diagnosis_mapping)
long_format_df['gender'] = long_format_df['temporary'].str[7].map({'m': 'male', 'f': 'female'})
long_format_df['age'] = long_format_df['temporary'].str[8:].map({
    '014': '0-14',
    '1524': '15-24',
    '2534': '25-34',
    '3544': '35-44',
    '4554': '45-54',
    '5564': '55-64',
    '65': '65+'
})

# droping the temporary column
long_format_df = long_format_df.drop(columns=['temporary'])

long_format_df.head()

,country,year,number_of_cases,diagnosis_method,gender,age
0,Afghanistan,1980,NaN,Pulmonary Smear (+),male,0-14
1,Afghanistan,1981,NaN,Pulmonary Smear (+),male,0-14
2,Afghanistan,1982,NaN,Pulmonary Smear (+),male,0-14
3,Afghanistan,1983,NaN,Pulmonary Smear (+),male,0-14
4,Afghanistan,1984,NaN,Pulmonary Smear (+),male,0-14


## Question 3: Create a graph that shows when countries started to report TB cases.

In [6]:
# filtering the rows where number of cases are not null
reporting_df = long_format_df[long_format_df['number_of_cases'].notnull()]

# finding the first year each country reported TB cases
first_report_year = reporting_df.groupby('country')['year'].min().reset_index()
first_report_year.columns = ['country', 'first_year']

# creating scatter plot
fig = px.scatter(first_report_year, x='first_year', y='country',
                 color='first_year',
                 labels={'first_year': 'Year', 'country': 'Country'},
                 title='Year When Countries Started Reporting TB Cases',
                 color_continuous_scale='viridis')

# display the plot
fig.show()